# Resultados
En este notebook nos encargamos de entrenar los modelos y mostrar resultados para cada uno de ellos. Mostraremos precisión, sensibilidad y F1 para cada categoría así como la media que estos ofrecen.

### Activamos Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importamos librerías necesarias



In [2]:
import pandas as pd
import numpy as np
import random
import math
import re
import warnings
import ipywidgets as widgets
from joblib import dump

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, naive_bayes
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

import nltk
from nltk.corpus import wordnet

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### Definimos función que nos dice si una palabra es un adjetivo, un nombre, un verbo o un advervio.

In [3]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

### Definimos función que recibe una cadena de texto y devuelve el texto procesado.

In [4]:
def preprocess(sentence):
    # Convertimos a minúsculas
    new_text = sentence.lower()
    
    # Eliminamos puntuación
    new_text = re.sub(r'[^\w\s]', '', new_text)

    # Dividimos en tokens
    tokens = nltk.tokenize.TreebankWordTokenizer().tokenize(new_text)

    # Eliminamos stopwords
    tokens = [word for word in tokens if not word in nltk.corpus.stopwords.words('english')]

    # Stemming
    # stemmer = nltk.stem.PorterStemmer()
    # new_text = ' '.join([stemmer.stem(w) for w in tokens])
    
    # lemma
    lemmatizer = nltk.stem.WordNetLemmatizer()
    new_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens])

    # new_text = ' '.join([w for w in tokens])

    # Reemplazamos números por #s
    if bool(re.search(r'\d', new_text)):
        new_text = re.sub('[0-9]{5,}', '#####', new_text)
        new_text = re.sub('[0-9]{4}', '####', new_text)
        new_text = re.sub('[0-9]{3}', '###', new_text)
        new_text = re.sub('[0-9]{2}', '##', new_text)
        # Cuando existe un solo número lo eliminamos
        new_text = re.sub('[0-9]{1}', '', new_text)

    return new_text

### Seleccionamos el modelo que queremos utilizar. Por defecto se utilizará SVM.

In [5]:
model_list = [
    ('SVM núcleo lineal', 0), 
    ('K-NN con k=1', 1), 
    ('Clasificador bayesiano ingenuo', 2), 
    ('Bosque aleatorio', 3)
]
model_picker = widgets.Dropdown(options=model_list)
print("Selecciona un modelo: ")
model_picker

Selecciona un modelo: 


Dropdown(options=(('SVM núcleo lineal', 0), ('K-NN con k=1', 1), ('Clasificador bayesiano ingenuo', 2), ('Bosq…

### Cargamos y preprocesamos la base de datos

In [6]:
# Cargamos base de datos
RUTA_DB = "./drive/My Drive/nfr-extraction/db.xlsx"
db = pd.read_excel(RUTA_DB)

# Definimos categorías existentes
categories = [
    "access control", "audit", "availability", "legal", "look and feel",
    "maintainability", "operational", "privacy", "recoverability", "capacity and performance",
    "reliability", "security", "usability", "other nonfunctional", "functional", "not applicable"
]
# Creamos diccionario con las traducciones para cada categoría
cat_translations =[
    "Control de acceso", "Auditoría", "Disponibilidad", "Legal", "Diseño",
    "Mantenibilidad", "Operacional", "Privacidad", "Recuperabilidad", "Rendimiento",
    "Fiabilidad", "Seguridad", "Usabilidad", "Otros no funcionales", "Funcional", "No aplicable"    
]

# Preprocesamos los elementos de la base de datos y barajamos de forma aleatoria
db['sentences'] = db['sentences'].apply(preprocess)
db = db.dropna()

db = db.sample(frac=1, random_state = 42).reset_index(drop=True)

### Entrenamos modelo, validamos y mostramos resultados de precisión, sensibilidad y F1

In [7]:
# Suprimimos posibles alertas
warnings.filterwarnings("ignore") 

# Definimos lista con los posibles modelos
models = [svm.LinearSVC(), KNeighborsClassifier(n_neighbors=1), naive_bayes.MultinomialNB(), RandomForestClassifier(n_jobs=-1)]
# Escogemos el modelo que vamos a utilizar
model = models[model_picker.value]

# Definimos variables necesarias para el almacenamiento de los resultados
k = 10
mean_precision = [[0]*16]*k
mean_recall = [[0]*16]*k
mean_f1score = [[0]*16]*k
i_test = 0 
f_test = math.floor(len(db)/k)

# Iteramos k veces (Validación cruzada con k = 10)
for index_k in range(k):
    print("\nIteración", index_k+1, end='')
    # Definimos variables de precisión, sensibilidad y F1 para esta iteración 
    precision = [0]*len(categories)
    recall = [0]*len(categories)
    f1score = [0]*len(categories)

    # Dividimos en datos de entrenamiento y validación
    train = db.drop(range(i_test, f_test))
    test = db[i_test:f_test]

    # Instanciamos el modelo que se encargará de aplicar tfidf a las frases
    vectorizer = TfidfVectorizer()
    # Lo entrenamos con el conjunto de datos de entrenamiento
    vectorizer.fit(list(train['sentences'].values))

    # Dividimos en vectores de características de entrenamiento y validación,
    # y clasificaciones para entrenamiento y validaciób
    x_train = vectorizer.transform(list(train['sentences'].values))
    y_train = train.drop(labels=['sentences'], axis=1)
    x_test = vectorizer.transform(list(test['sentences'].values))
    y_test = test.drop(labels=['sentences'], axis=1)

    i = 0
    # Para cada categoría
    for category in categories:
        print('.', end='.')
        # Entrenamos el modelo
        model.fit(x_train, y_train[category])
        # Realizamos predicción para conjunto de prueba
        prediction = model.predict(x_test)
        # Obtenemos resultados
        precision[i] += precision_score(list(y_test[category].values), prediction)
        recall[i] += recall_score(list(y_test[category].values), prediction)
        f1score[i] += f1_score(list(y_test[category].values), prediction)
        i += 1
    
    i_test = f_test
    f_test = i_test + math.floor(len(db)/k)
    
    mean_precision[index_k] = precision
    mean_recall[index_k] = recall
    mean_f1score[index_k] = f1score

# Mostramos resultados agrupados en un dataframe de pandas
p = list(np.average(mean_precision, axis=0))
rounded_p = [round(elem, 2) for elem in p]
r = list(np.average(mean_recall, axis=0))
rounded_r = [round(elem, 2) for elem in r]
f1 = list(np.average(mean_f1score, axis=0))
rounded_f1 = [round(elem, 2) for elem in f1]
data = {
    " ": cat_translations + ['MICRO AVG'],
    "Precisión": rounded_p + [np.average(rounded_p)],
    "Sensibilidad": rounded_r + [np.average(rounded_r)],
    "F1": rounded_f1 + [np.average(rounded_f1)]
}
print("\nLos resultados para el modelo seleccionado son:")
pd.options.display.float_format = "{:,.2f}".format
pd.DataFrame(data).set_index(' ')



Iteración 1................................
Iteración 2................................
Iteración 3................................
Iteración 4................................
Iteración 5................................
Iteración 6................................
Iteración 7................................
Iteración 8................................
Iteración 9................................
Iteración 10................................
Los resultados para el modelo seleccionado son:


,Precisión,Sensibilidad,F1
,,,
Control de acceso,0.80,0.64,0.71
Auditoría,0.80,0.46,0.58
Disponibilidad,0.68,0.35,0.44
Legal,0.87,0.46,0.60
Diseño,0.80,0.36,0.48
Mantenibilidad,0.84,0.46,0.59
Operacional,0.70,0.37,0.48
Privacidad,0.79,0.46,0.58
Recuperabilidad,0.76,0.37,0.48


### Exportamos los modelos
Los modelos que exportaremos coincidiran con los entrenados en la primera
iteración de la validación cruzada. De esta manera podremos utilizar estos modelos en otro lugar para poder clasificar nuevos elementos. También debemos exportar el modelo (TfidfVectorizer) utilizado para convertir las frases en vectores de números.

In [8]:
# Dividimos en datos de entrenamiento y validación
train = db.drop(range(0, math.floor(len(db)/k)))
test = db[0:math.floor(len(db)/k)]

# Instanciamos Vectorizador TFIDF
vectorizer = TfidfVectorizer()
# Entrenamos con los elementos de entrenamiento
vectorizer.fit(list(train['sentences'].values))
# Exportamos el vectorizador
dump(vectorizer, './drive/My Drive/nfr-extraction/models/vectorizer.joblib')

# Obtenemos X e Y para conjunto de entrenamiento y de validación
x_train = vectorizer.transform(list(train['sentences'].values))
y_train = train.drop(labels=['sentences'], axis=1)
x_test = vectorizer.transform(list(test['sentences'].values))
y_test = test.drop(labels=['sentences'], axis=1)

# Para cada categoría
for category in categories[:-1]:
    model = models[model_picker.value]
    model.fit(x_train, y_train[category])
    dump(model, './drive/My Drive/nfr-extraction/models/' + category.replace(" ", "") + '.joblib')
